# Dreambooth: Generating Stable Diffusion Portaits

    This notebook is based on the work of Laurence Day and the "The Repo Formerly Known As Dreambooth" by Joe Penna and his many extraordinarily talented collaborators, and further tweaks to the training process have been made by Kane Wallmann. 
    
    Over the course of a first run of this repository, approximately 5.2GB will be downloaded (excluding packages).
    You won't have to worry about those downloads though, they happen automatically, just note that they take time.
    
    There may well be long stretches of time where it looks to you as if nothing is happening (i.e. no images are
    appearing in the /outputs directory). If the icon for the tab that this notebook is running in is an orange 
    hour-glass, things are still grinding along. If it's changed back to an orange book, then it's done.
    
    While you're running things, you can scroll down and see which cells have 'completed' and which are either running
    or are still scheduled to run. To the left of each cell is a little [] icon - if it looks like [*] it's still queued,
    whereas if it's got a number in it like [5] then it has already completed.
    
    Some provisos:
        * I have bundled Stable Diffusion 1.4 in the repository containing this notebook. Please read the license 
          file `STABLE DIFFUSION LICENSE.md` in order to understand what you implicitly agree to by making use of this tech.
          The long and short of it is that so long as you don't approach this tool with ill intent, you're fine.
        * Despite Kane's adjustments enabling multi-subject training, this repo *at present* is designed to only accept 
          one subject at a time - that's what the prompts are expecting, so training multiple people is a waste. In future
          iterations of this repository, I will introduce some branching logic allowing you to select how many people you
          want to include in the generated images.

# Naming And Classification

\*\*\* This is the **only** cell you have to edit. \*\*\* 

Before you opened this notebook per instructions from the repository, you should have uploaded 8-10 .png photographs of the person you are trying to generate images for into the directory `/training_samples/{class}` with filenames:

* `name class_001.png`
* `name class_001.png`
* ...
* `name class_00n.png`

If you haven't done this, go back to the README of this repository (scroll down from the link you were given) and give it a go.

Your choices for class are `man`, `woman`, `person`, `dog` or `cat`, whereas name is dealers choice, but do keep it as a single word (i.e. no spaces).

**For example**, if you're trying to train on Princess Diana, your files would be called `princessdiana woman_001.png` et cetera.

**It is advisable to use a name choice so that - if you're called Willow, for example - the model doesn't think you're Willow from Buffy, or think you're Asian if your name is Jian. I recommend using a name such as `jianthisisdefinitelyme` if you have a name that could cause confusion.**

The spacing and underscores are important here, so please double-check.

In the cell below, edit the variables to reflect the `name` and `class` you have chosen. _Again_, please check for accuracy.

In [ ]:
target_name = "princessdiana"

target_class = "woman"

### If you do not want to adjust any of the optional parameters (see below)...

# You can now click *Cell > Run All* from the menu bar above.

## Go away for 90 minutes.

---

## Optional Parameters

The `run_length` variable dictates how many images are generated for each prompt.

The default, `medium`, produces 64 images per prompt, but should you choose, you can edit this to either:

* `short` - _25_ images per prompt
* `medium` - _64_ images per prompt
* `long` - _100_ images per prompt

Note that shortening the run length will produce results faster, but give you fewer results from which to select a true likeness, whereas lengthening it will slow things down by a fair margin, but produce a much wider choice at the end.


In [ ]:
run_length = "short"

# Build Environment

In [ ]:
import os

prereq_pass_file = '/workspace/Dreambooth-Ion-Cannon/install_done'
packages_installed = os.path.exists(prereq_pass_file)
print('Packages installed?: ' + str(packages_installed))

if (not packages_installed):
    #BUILD ENV
    !pip install omegaconf
    !pip install einops
    !pip install pytorch-lightning==1.6.5
    !pip install test-tube
    !pip install transformers
    !pip install kornia
    !pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
    !pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
    !pip install setuptools==59.5.0
    !pip install pillow==9.0.1
    !pip install torchmetrics==0.6.0
    !pip install -e .
    !pip install protobuf==3.20.1
    !pip install gdown
    !pip install pydrive
    !pip install -qq diffusers["training"]==0.3.0 transformers ftfy
    !pip install -qq "ipywidgets>=7,<8"
    !pip install ipywidgets==7.7.1
    !pip install py7zr==0.20.0
    !pip install multivolumefile

    # Lazy way of marking installation completion to save time on Cell > Run All repeats
    finished = open('install_done', 'w+')
    finished.close()

# Downloading/Reconstituting Stable Diffusion v1.4

In [ ]:
model_path = '/workspace/Dreambooth-Ion-Cannon/model.ckpt'
valid_model_size = 4265380512 # doing this instead of SHA256 checksum, it's quicker

already_exists = os.path.exists(model_path)
print('Does model.ckpt exist: ' + str(already_exists))
correct_model_size = False

if (already_exists):
    model_size = os.path.getsize(model_path)
    print('model.ckpt size in bytes: ' + str(model_size))
    if (model_size == valid_model_size):
        print('No need to (re)download, model already present')
        correct_model_size= True

if ((not already_exists) or (not correct_model_size)):
    !git clone https://github.com/laurenceday/Stable-Diffusion-1.4.git
    # TODO We may want to save our own copy of stable diffusion

    import multivolumefile
    import py7zr
    with multivolumefile.open('/workspace/Dreambooth-Ion-Cannon/Stable-Diffusion-1.4/model.7z', mode='rb') as target_archive:
        with py7zr.SevenZipFile(target_archive, 'r') as archive:
            archive.extractall()
    
    # Remove repository and contents so git doesn't complain if it needs to rerun
    !rm -rf '/workspace/Dreambooth-Ion-Cannon/Stable-Diffusion-1.4'

# Retrieving Regularisation Images

In [ ]:
!git clone https://github.com/laurenceday/Ion-Cannon-Regularisation-{target_class}.git

# TODO If we want to replace regularisation images, we can create new directories here
!mkdir -p regularisation_images/{target_class}
!mv -v Ion-Cannon-Regularisation-{target_class}/{target_class}/*.* regularisation_images/{target_class}

# Training and Pruning

In [ ]:
target_token = f"{target_name} {target_class}"
project_name = f"{target_name}_model"
model_name = f"{project_name}.ckpt"

In [ ]:
!rm -rf training_samples/{target_class}/.ipynb_checkpoints
!python "main.py" \
 --base configs/stable-diffusion/v1-finetune_unfrozen.yaml \
 -t \
 --actual_resume "model.ckpt" \
 --reg_data_root "/workspace/Dreambooth-Ion-Cannon/regularisation_images" \
 -n {project_name} \
 --gpus 0, \
 --data_root "/workspace/Dreambooth-Ion-Cannon/training_samples" \
 --max_training_steps 1000 \
 --no-test

In [ ]:
directory_paths = !ls -d logs/*

In [ ]:
# This version should automatically prune around 10GB from the ckpt file
last_checkpoint_file = directory_paths[-1] + "/checkpoints/last.ckpt"
!python "prune_ckpt.py" --ckpt {last_checkpoint_file}

In [ ]:
last_checkpoint_file_pruned = directory_paths[-1] + "/checkpoints/last-pruned.ckpt"
!mkdir -p trained_models
!mv {last_checkpoint_file_pruned} trained_models/{model_name}
# TODO if we want to save the model on a server somewhere, this is the file we should save

In [ ]:
model_path = f"/workspace/Dreambooth-Ion-Cannon/trained_models/{model_name}"

# Image Generation Variable Setup

In [ ]:
if (run_length == "short"):
    sample_n = 5
elif (run_length == "long"):
    sample_n = 10
else:
    sample_n = 8

# Regular Image Generation

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples 2 \
 --n_iter 2 \
 --seed 42 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "{target_token}" \
 --outloc "{target_name}_raw" \
 --outdir "outputs/{target_name}_raw"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples {sample_n} \
 --n_iter {sample_n} \
 --seed 1201562208 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "digital painting of {target_token} by greg rutkowski neon cyberpunk" \
 --outloc "{target_name}_cyberpunk" \
 --outdir "outputs/{target_name}_cyberpunk"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples {sample_n} \
 --n_iter {sample_n} \
 --seed 1337 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "{target_token} cosmic background realistic shaded perfect face, fine details by realistic shaded lighting poster by ilya kuvshinov katsuhiro otomo, magali villeneuve, artgerm, jeremy lipkin and michael garmash and rob rey" \
 --outloc "{target_name}_cosmic2" \
 --outdir "outputs/{target_name}_cosmic2"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples {sample_n} \
 --n_iter {sample_n} \
 --seed 1694397999 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "a beautiful portrait of {target_token}, with isolated flowers with strong dark comic outlines, colorful, psychedelic, intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, art by artgerm and greg rutkowski and alphonse mucha" \
 --outloc "{target_name}_flowers" \
 --outdir "outputs/{target_name}_flowers"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples {sample_n} \
 --n_iter {sample_n} \
 --seed 1694397999 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "up close portrait of a beautiful {target_token} in black and white, photorealistic, art by diego fazio and diegoKoi and oscar Ukono, concept art, sharp focus, artgerm, 8k highly detailed" \
 --outloc "{target_name}_blackwhite" \
 --outdir "outputs/{target_name}_blackwhite"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples {sample_n} \
 --n_iter {sample_n} \
 --seed 1033162293 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "Ultra realistic medium shot of a cyborg with face of {target_token}, cyberpunk, sci - fi, kodak, faces, colour led, soft light, volumetric lighting, fog, rays, night, intricate detailed, digital painting, concept art, smooth, sharp focus, illustration, art by artgerm and greg rutkowski and alphonse mucha" \
 --outloc "{target_name}_cyborg" \
 --outdir "outputs/{target_name}_cyborg"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples {sample_n} \
 --n_iter {sample_n} \
 --seed 1694397999 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "a colored pencil portrait of {target_token} standing in a backyard with falling leaves, autumn, red, yellow, orange" \
 --outloc "{target_name}_autumn" \
 --outdir "outputs/{target_name}_autumn"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples {sample_n} \
 --n_iter {sample_n} \
 --seed 2283716726 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "Portrait of a beautiful {target_token}, intricate, elegant, highly detailed, occult imagery, digital painting, artstation, concept art, illustration, by gil elvgen, greg manchess, mucha" \
 --outloc "{target_name}_paint" \
 --outdir "outputs/{target_name}_paint"